# convert old version to teacher/kid format

In [2]:
import pandas as pd
import os
import numpy as np

In [6]:
old_pred_dir = r'D:\ShanghaiASD_project\gazefollow_pattern_checkup\pred_stats\20230803_v0918'


In [7]:
kid_id = '109'
pred_dir = '%s/%s'%(old_pred_dir, kid_id)
cams = os.listdir(pred_dir)


['109小鸟', '109瓢虫1号', '109瓢虫2号', '109考拉', '109袋鼠']

In [22]:
cleaned_gt_dir = r'D:\ShanghaiASD_project\gazefollow_pattern_checkup\ENCU_base_annotations\cleaned'

activity_dir = r'D:\ShanghaiASD_project\gazefollow_pattern_checkup\activity_mapping'
a_file = '%s/%s_activity.csv'%(activity_dir, kid_id)
a_df = pd.read_csv(a_file)

gt_df = pd.read_csv('%s/%s.txt'%(cleaned_gt_dir, kid_id))

stats_header = ['camera', 'activity', 'acc', 'recall', 'precision']
pd_stats = []


for cam in os.listdir(pred_dir):
    pred_cam_df = pd.read_excel('%s/%s/activity_summary_kid.xlsx'%(pred_dir, cam))
    gt_kid_at_teacher = gt_df[gt_df.SXC=='face'].dropna(subset=['activity'])
    for i in range(len(a_df)):
        start_f, end_f, act = a_df.iloc[i]
        total_frames = range(start_f, end_f)
        a_gt = gt_kid_at_teacher[gt_kid_at_teacher.activity==act]
        a_pred = pred_cam_df[pred_cam_df.activity==act]
        acc_score, recall, precision = obtain_score_per_act(a_pred, a_gt)
#         print("At activity: %s, Acc: %.2f, Recall: %.2f, Precision: %.2f"%(act, acc_score*100, recall*100, precision*100))
        pd_stats.append([cam, act, acc_score*100, recall*100, precision*100])


In [23]:
pd.DataFrame(pd_stats, columns=stats_header)

,camera,activity,acc,recall,precision
0,109小鸟,Bamboo,74.882471,75.0,5.412371
1,109小鸟,Puzzle1,59.974384,100.0,0.398406
2,109小鸟,RJAposterA,100.000000,NaN,NaN
3,109小鸟,RJAposterB,86.218487,NaN,0.000000
4,109小鸟,RJAduck,56.680162,NaN,0.000000
...,...,...,...,...,...
60,109袋鼠,RJAposterC,25.471698,NaN,0.000000
61,109袋鼠,RJAposterD,36.551724,NaN,0.000000
62,109袋鼠,RJAcar,28.801431,NaN,0.000000
63,109袋鼠,IJAcar,86.380114,NaN,0.000000


In [24]:
def collect_lists(gt_df, pred_df):
    gt_ls = []
    for i in gt_df.index:
        start_f = gt_df.at[i, 'start_frame']
        end_f = gt_df.at[i, 'end_frame']
        for f in range(start_f, end_f):
            gt_ls.append(f)
    
    pred_ls = []
    for j in pred_df.index:
        if 'teacher' in pred_df.at[j, 'pattern']:
            f = pred_df.at[j, 'frameID']
            pred_ls.append(f)
    return gt_ls, pred_ls
            
def obtain_score(a_pred, a_gt, total_frames):
    gt_ls, pred_ls = collect_lists(a_gt, a_pred)        
    tp_ls = []
    fp_ls = []
    fn_ls = []
    tn_ls = []
    for f in total_frames:
        if f in pred_ls: # a predicted positive
            if f in gt_ls: # true positive
                tp_ls.append(f)
            else: # false positive
                fp_ls.append(f)
        else: # a predicited negative
            if f in gt_ls: # false negative
                fn_ls.append(f)
            else: # true negative
                tn_ls.append(f)
    assert len(tp_ls)+len(fp_ls)+len(fn_ls)+len(tn_ls)==len(total_frames)
    acc_score = (len(tp_ls)+len(tn_ls))/len(total_frames)
    
    if len(gt_ls)==0:
        recall = np.nan
    else:
        recall = len(tp_ls)/(len(tp_ls)+len(fn_ls))
    if len(pred_ls)==0:
        precision = np.nan
    else:
        precision = len(tp_ls)/(len(tp_ls)+len(fp_ls))
    
    return acc_score, recall, precision

In [16]:
pred_cam_df.pattern.unique()

array(['void', 'at_bamboo', 'at_teacher', 'at_duck', 'at_duck/at_bamboo',
       'at_car', 'at_car/at_bamboo'], dtype=object)